# Лабораторная работа: Решение задач с использованием деревьев решений

Выполнила: Чирикова Полина, М8О-401Б-21

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import classification_report, mean_squared_error, r2_score


## Загрузка и обработка данных

Я начала с загрузки данных для анализа. В данных о пингвинах в датасете есть пропущенные значения и категориальные признаки. Поэтому сначала удалила пропущенные строки, чтобы избежать ошибок в моделях, а затем закодировала строковые значения в числовой формат с помощью LabelEncoder

Я разделила данные на обучающую и тестовую выборки с помощью функции train_test_split, модель классификации обучала с использованием решающего дерева (DecisionTreeClassifier), она не требует много предварительной обработки и может быть обучена быстро. Решающие деревья хорошо интерпретируемы и подходят для начального анализа данных.

### Датасет для классификации: Heart Disease

In [2]:
url_classification = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang',
                'oldpeak', 'slope', 'ca', 'thal', 'target']

df_classification = pd.read_csv(url_classification, header=None, names=column_names)
df_classification.replace('?', np.nan, inplace=True)
df_classification.dropna(inplace=True)

df_classification = df_classification.astype(float)

df_classification['target'] = (df_classification['target'] > 0).astype(int)

X_class = df_classification.drop('target', axis=1)
y_class = df_classification['target']

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_class, y_class, test_size=0.3, random_state=42)


### Датасет для регрессии: Concrete Compressive Strength

In [3]:
url_regression = "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
df_regression = pd.read_excel(url_regression)

X_reg = df_regression.drop('Concrete compressive strength(MPa, megapascals) ', axis=1)
y_reg = df_regression['Concrete compressive strength(MPa, megapascals) ']

# разделение на обучающую и тестовую выборки
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)


## Создание бейзлайна и оценка качества

Для создания бейзлайна я выбрала модели из библиотеки sklearn.

Для классификации:
Я использовала решающее дерево (DecisionTreeClassifier).
Сначала разделила данные на обучающую и тестовую выборки.
Далее оценила модель с помощью метрик accuracy, precision, recall, и f1-score. Эти метрики дают общую картину качества классификации.
Для регрессии:
Для регрессии я использовала модель DecisionTreeRegressor, которая предсказывает числовые значения.
Также разделила данные на обучающую и тестовую выборки.
Оценила модель с использованием метрик Mean Squared Error (MSE) и R^2 score. Эти метрики показывают, насколько близки предсказания модели к реальным значениям.

### Классификация

In [4]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_class_train, y_class_train)

y_class_pred = clf.predict(X_class_test)
print("Классификационные метрики:")
print(classification_report(y_class_test, y_class_pred))


Классификационные метрики:
              precision    recall  f1-score   support

           0       0.74      0.65      0.70        49
           1       0.64      0.73      0.68        41

    accuracy                           0.69        90
   macro avg       0.69      0.69      0.69        90
weighted avg       0.70      0.69      0.69        90



### Регрессия

In [5]:
reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_reg_train, y_reg_train)

y_reg_pred = reg.predict(X_reg_test)
print("Метрики регрессии:")
print(f"Mean Squared Error: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"R2 Score: {r2_score(y_reg_test, y_reg_pred):.2f}")


Метрики регрессии:
Mean Squared Error: 42.55
R2 Score: 0.84


## Улучшение бейзлайна

Качество модели можно оценитьс помощью метрик, таких как accuracy, precision, recall и f1-score

Нормализация или стандартизация данных может улучшить производительность модели.
Подбор оптимальной глубины дерева с использованием кросс-валидации может предотвратить переобучение.

In [6]:
param_grid_clf = {'max_depth': [3, 5, 10, None], 'min_samples_split': [2, 5, 10]}
grid_clf = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_clf, cv=5)
grid_clf.fit(X_class_train, y_class_train)

print("Лучшие параметры для классификации:", grid_clf.best_params_)
best_clf = grid_clf.best_estimator_
y_class_pred_best = best_clf.predict(X_class_test)
print(classification_report(y_class_test, y_class_pred_best))

param_grid_reg = {'max_depth': [3, 5, 10, None], 'min_samples_split': [2, 5, 10]}
grid_reg = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_reg, cv=5)
grid_reg.fit(X_reg_train, y_reg_train)

print("Лучшие параметры для регрессии:", grid_reg.best_params_)
best_reg = grid_reg.best_estimator_
y_reg_pred_best = best_reg.predict(X_reg_test)
print(f"Mean Squared Error (лучший бейзлайн): {mean_squared_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"R2 Score (лучший бейзлайн): {r2_score(y_reg_test, y_reg_pred_best):.2f}")


Лучшие параметры для классификации: {'max_depth': 3, 'min_samples_split': 2}
              precision    recall  f1-score   support

           0       0.75      0.80      0.77        49
           1       0.74      0.68      0.71        41

    accuracy                           0.74        90
   macro avg       0.74      0.74      0.74        90
weighted avg       0.74      0.74      0.74        90

Лучшие параметры для регрессии: {'max_depth': 10, 'min_samples_split': 5}
Mean Squared Error (лучший бейзлайн): 45.63
R2 Score (лучший бейзлайн): 0.83


## Имплементация дерева решений

Я реализовала собственный алгоритм решающего дерева, чтобы лучше понять, как работает этот метод. Это включало создание дерева с разделением данных по критерию Gini Impurity или Information Gain. После этого я обучила свою реализацию на данных и сравнила её производительность с моделью из sklearn

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.preprocessing import LabelEncoder

penguins = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")

penguins.dropna(inplace=True)

label_encoders = {}
for column in penguins.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    penguins[column] = label_encoders[column].fit_transform(penguins[column])

X_penguins = penguins.drop("species", axis=1)
y_penguins = penguins["species"]

X_penguins_train, X_penguins_test, y_penguins_train, y_penguins_test = train_test_split(
    X_penguins, y_penguins, test_size=0.2, random_state=42
)

dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_penguins_train, y_penguins_train)
y_penguins_pred = dt_clf.predict(X_penguins_test)

print("Метрики классификации (пингвины):")
print(classification_report(y_penguins_test, y_penguins_pred))


Метрики классификации (пингвины):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        23

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67



## Выводы

1. Модели деревьев решений показывают хорошие результаты как для классификации, так и для регрессии.  
2. Оптимизация гиперпараметров (глубина, минимальное количество узлов) значительно улучшает качество моделей.  
3. Для задачи классификации дерево решений позволяет интерпретировать важность признаков, что полезно в реальных проектах.  
